In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import iqr
# from nsepython import *

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# from bsedata.bse import BSE
# b = BSE(update_codes = True)
# data = b.getScripCodes()

In [3]:
# # Initialize an empty list to store data
# rows = []

# # Assuming b is an object with a method getQuote() to retrieve information for a given code
# for code, company in data.items():
#     try:
#         q = b.getQuote(code)
#         row = {'Code': code,
#                'Company': company,
#                'SecurityID': q.get('securityID', None),
#                'MarketCapFull': q.get('marketCapFull', None),
#                'MarketCapFreeFloat': q.get('marketCapFreeFloat', None),
#                'Industry': q.get('industry', None)}
#         rows.append(row)
#     except Exception as e:
#         print(f"Error fetching data for code {code}: {e}")

# # Convert the list of dictionaries to a DataFrame
# df = pd.DataFrame(rows)
# df.to_csv('stock_data.csv')

In [4]:
df = pd.read_csv('stock_data.csv')
df.set_index('Code', inplace=True)
df['MarketCapFull'] = pd.to_numeric(df['MarketCapFull'].str.replace(',', '').str.replace(' Cr.', ''), errors='coerce')
df = df[df['MarketCapFull'] > 200]

In [5]:
%%capture

high_volume_stocks = pd.DataFrame({'SYMBOL': [], 'FIRST_HIGH_VOLUME': [], 
                                    'LAST_HIGH_VOLUME': [], 'HIGH_VOL_COUNT': [],
                                    'MARKET_CAP':[], 'INDUSTRY': [], 'DOWN':[]})

for index, row in df.iterrows():
    try:
        stock = yf.download(row['SecurityID'] + ".BO", start=(pd.to_datetime('today') - pd.DateOffset(days=180)), end=(pd.to_datetime('today') + pd.DateOffset(days=1)), progress=False)
        stock.columns = [col[0] for col in stock.columns]
        iqr_stock = iqr(stock['Volume'])
        volume_cutoff = np.quantile(stock['Volume'], 0.75) + 5 * iqr_stock ## To find Outliers
        high_volume_stock = stock[stock['Volume'] > volume_cutoff]
        down_from_high = round((stock['Close'][-1] / stock['Close'].max())*100, 1)
        
        # convert this data into a row
        if(high_volume_stock.size > 0):
            new_stock = pd.DataFrame({'SYMBOL': [row['SecurityID']], 
                                      'FIRST_HIGH_VOLUME': [high_volume_stock.index[0]], 
                                      'LAST_HIGH_VOLUME': [high_volume_stock.index[high_volume_stock.shape[0]-1]], 
                                      'HIGH_VOL_COUNT': [high_volume_stock.shape[0]],
                                      'MARKET_CAP':[row['MarketCapFull']],
                                      'INDUSTRY':[row['Industry']],
                                      'DOWN':down_from_high})
            high_volume_stocks = pd.concat([high_volume_stocks, new_stock], ignore_index=True)
    except Exception as e:
        print(f"Error processing data for symbol {row['SecurityID']}: {e}")

print(f"Execution completed with {high_volume_stocks.shape[0]} stocks")

In [6]:
high_volume_stocks_recent = high_volume_stocks[high_volume_stocks['LAST_HIGH_VOLUME'] > (pd.to_datetime('today') - pd.DateOffset(days=10))]

In [7]:
# DOWN FILTER
high_volume_stocks_recent = high_volume_stocks_recent[high_volume_stocks_recent['DOWN'] >= 70]

In [8]:
high_volume_stocks_recent.set_index('SYMBOL', inplace=True)
high_volume_stocks_recent.head()

In [9]:
def download_stock_data(symbol):
    return yf.download(symbol + ".BO", start=pd.to_datetime('today') - pd.DateOffset(365),
                       end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)

def calculate_technical_indicators(data):
    data['5DMA'] = data['Close'].rolling(window=5).mean()
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['100DMA'] = data['Close'].rolling(window=100).mean()
    data['diff'] = data['Close'] - data['Open']
    data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    return data

def plot_stock_data(symbol, data):
    # Prepare plot data
    plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=180))]

    # Setup subplot layout
    figure = make_subplots(specs=[[{"secondary_y": True}]])

    # Plot candlestick chart
    figure.add_trace(go.Candlestick(x=plot_data.index,
                                    open=plot_data['Open'],
                                    high=plot_data['High'],
                                    low=plot_data['Low'],
                                    close=plot_data['Close'],
                                    name='Price'))

    # Adjust Y-axis and layout for stock prices
    figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05])
    figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])], row=1, col=1)  # hide weekends
    figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)

    # Plot moving averages and volume
    for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
        figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data[ma], marker_color=color, name=f'{ma}'))
    figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume',
                            marker={'color': plot_data['color']}), secondary_y=True)
    

    # Hide the secondary Y-axis (volume)
    figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True)
    figure.update_yaxes(visible=False, secondary_y=True)
    return figure

def calculate_stock_characteristics(symbol):
    stock_row = high_volume_stocks_recent.loc[symbol]
    characteristics = pd.DataFrame({
                "Attribute": high_volume_stocks_recent.columns.values[1:],
                "Value": stock_row.values[1:],
            }).set_index("Attribute").T
    return characteristics


def process_stocks(all_symbols):
    num_processed = 0
    file_count = 1
    while num_processed < len(all_symbols):
        figure_html = open(f'spike_in_volume_stocks_{file_count}.html', 'w')
        for symbol in all_symbols[num_processed:num_processed+100]:
            try:
                data = download_stock_data(symbol)
                data = calculate_technical_indicators(data)
                figure = plot_stock_data(symbol, data)
                figure_html.write(figure.to_html(full_html=False))
            except Exception as e:
                print(f"Error processing symbol {symbol}: {e}")
        figure_html.close()
        num_processed += 100
        file_count += 1


if __name__ == "__main__":
    symbols = high_volume_stocks_recent.index.values
    process_stocks(symbols)
    print('Execution Completed!!!!')